In [1]:
import os
import numpy as np
env_var = os.environ
os.environ["LACE_REPO"] = "/nfs/pic.es/user/l/lcabayol/DESI/LaCE"
os.environ["LACE_MANAGER_REPO"] = "/nfs/pic.es/user/l/lcabayol/DESI/LaCE_manager"

In [2]:
sys.path.append('emulator.py')

In [3]:
from emulator import P1Demulator
from lace.emulator import pd_archive
from lace.emulator.test_simulation_nn import test_sim


 /data/astro/scratch/lcabayol/anaconda3/envs/DESIenv6/lib/python3.10/site-packages/lace-1.0.1-py3.10.egg/lace/emulator/pd_archive.py:210: SyntaxWarning:"is" with a literal. Did you mean "=="?
 /data/astro/scratch/lcabayol/anaconda3/envs/DESIenv6/lib/python3.10/site-packages/lace-1.0.1-py3.10.egg/lace/emulator/pd_archive.py:210: SyntaxWarning:"is" with a literal. Did you mean "=="?
 /data/astro/scratch/lcabayol/anaconda3/envs/DESIenv6/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning:IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html


In [4]:
# our modules
from lace.cosmo import fit_linP
from lace.emulator import poly_p1d


In [5]:
kmax_Mpc=4
ndeg=5

## Neural Network

In [10]:
emulator = P1Demulator(save_path=f'/nfs/pic.es/user/l/lcabayol/DESI/LaCE/lace/emulator/NNmodels/NNEmulator_LaCEHC.pt')

start the training of the emulator
Training network on 11550
Emualtor trained in 107.70529174804688 seconds


In [11]:
truth=pd_archive.archivePD(z_max=4.5,nsamples=30, pick_sim='central')
truth.average_over_samples(flag="all")
zs = [d['z'] for d in truth.data_av_all if d['scale_tau'] == 1] 
truth = [d for d in truth.data_av_all if d['scale_tau'] == 1] 

fractional_error, emu_p1d, true_p1d, emu_p1derr = test_sim(emulator.emulator.emulator, truth, emulator.emulator.emuparams, emulator.emulator.paramLims, emulator.emulator.device, emulator.emulator.yscalings,  kmax_Mpc_test=kmax_Mpc, ndeg=ndeg )


42
Mean fractional error: 0.9948371740004323
Std fractional error: 0.004675616305180939


## Gaussian Process

In [6]:
emulator = P1Demulator(emu_algorithm='GP',  list_archives=['data_av_all'])


KeyboardInterrupt



In [9]:
truth=pd_archive.archivePD(z_max=4.5, pick_sim='central')
truth.average_over_samples(flag="all")
truth = [d for d in truth.data_av_all if d['scale_tau'] == 1] 

for aa,item in enumerate(truth):
    # figure out redshift for this entry
    z=item["z"]

    true_k=item["k_Mpc"]
    k_mask=(true_k<kmax_Mpc) & (true_k>0)
    true_p1d=item["p1d_Mpc"][k_mask]
    coeff_p1d = poly_p1d.PolyP1D(true_k[k_mask],true_p1d,kmin_Mpc=1.e-3,kmax_Mpc=emulator.emulator.kmax_Mpc,deg=emulator.emulator.ndeg).lnP_fit
    poly=np.poly1d(coeff_p1d)
    true_p1d_poly =np.exp(poly(np.log(true_k[k_mask])))

    # true p1d (some sims have an extra k bin, so we need to define mask again)
    true_k=item["k_Mpc"]
    k_mask=(true_k<kmax_Mpc) & (true_k>0)
    true_p1d=item["p1d_Mpc"][k_mask]
    assert len(true_p1d)==(emulator.emulator.k_bin-1)


    # for each entry, figure emulator parameter describing it (labels)
    emu_call={}
    for bb,param in enumerate(emulator.emulator.paramList):
        emu_call[param]=item[param]

    # ask emulator to emulate P1D (and its uncertainty)
    emu_p1d,emu_err=emulator.emulator.emulate_p1d_Mpc(emu_call,emulator.emulator.training_k_bins,return_covar=True)
    fractional_errors_gp = emu_p1d/true_p1d_poly
print(f'mean fractional error: {np.mean(fractional_errors_gp)}')
print(f'std fractional error: {np.std(fractional_errors_gp)}')

mean fractional error: 0.9909885143284279
std fractional error: 0.004350958265193714


## Test old post-processing

In [12]:
emulator = P1Demulator(postprocessing='500')

start the training of the emulator
Training network on 11550
Emualtor trained in 107.57688426971436 seconds


In [13]:
emulator = P1Demulator(postprocessing='500',emu_algorithm='GP',  list_archives=['data_av_all'])

Training GP on 330 points
GPs optimised in 0.45 seconds


In [6]:
emulator = P1Demulator(postprocessing='1600',emu_algorithm='GP',  list_archives=['data_av_all'])

Exception: Available post-processings are "500" and "768"